In [1]:
import pandas as pd
import numpy as np
import re
from itertools import groupby
import math
import heapq
import pickle

In [2]:
%%time
# df = pd.read_excel("../IR_Spring2021_ph12_7k.xlsx")
df = pd.read_csv("IR_Spring2021_ph12_7k.csv")

CPU times: user 214 ms, sys: 29.1 ms, total: 244 ms
Wall time: 336 ms


In [3]:
def normalize(token):
    token = token.strip(".ء \u200b\u200c\u200d")
    token = re.sub("[ـ،؛,–—!٪؟+:_٫/.]", " ", token)
    token = re.sub("[*%&…#=\\\•;!|\-?]", " ", token)
    token = re.sub("[﴾﴿«»()<>\[\]“”'\"]", " ", token)
    token = re.sub("[\u200f\u200e\ufeff\u2067\u202a\u202b\u202c\u2069\xad]", " ", token)
    token = re.sub("\s+", " ", token)
    
    plurals = {
        "آداب": "ادب", "اطراف": "طرف", "حقایق": "حقیقت", "امواج": "موج",
        "مراکز": "مرکز", "اعماق": "عمق", "مواقع": "موقع", "اخبار": "خبر",
        "علما": "عالم", "آثار": "اثر", "مصارف": "مصرف", "علوم": "علم",
        "ادیان": "دین", "علائم": "علامت", "اسامی": "اسم", "مباحث": "مبحث",
        "دفاتر": "دفتر", "علل": "علت", "مذاهب": "مذهب", "عناصر": "عنصر",
        "مساجد": "مسجد", "روابط": "رابطه", "اعضا": "عضو", "عبارات": "عبارت",
        "موارد": "مورد", "مفاهیم": "مفهوم", "اشعار": "اشعار", "منابع": "منبع",
        "منبع": "قاعده", "فقها": "فقیه", "عجایب": "عجیب", "تصاویر": "تصویر"
    }
    
    for key, value in plurals.items():
        if key in token:
            token = token.replace(key, value)
            
    arabic_chars = ['\u064B', '\u064C', '\u064D', '\u064E', '\u064F', '\u0650', '\u0651', '\u0652']
    token = re.sub("|".join(arabic_chars), "", token)
    
    postfixes_a = ["ها", "های", "هایی", "ی", "ان",
                   "تر", "ترین",
                   "گر"]
    postfixes_b = ["\u200bها", "\u200cها", "\u200bهای", "\u200cهای", 
                   "\u200bهایی", "\u200cهایی",
                   "\u200bشده", "\u200cشده",
                   "\u200bساز", "\u200cساز",
                   "\u200bکنندگان", "\u200cکنندگان",
                   "\u200bتر", "\u200cتر", "\u200bترین", "\u200cترین"]
    
    stemming_b = ["می\u200b", "می\u200c", "نمی\u200b", "نمی\u200c"]
    stemming_e = ["ات", "ام", "اش", 
                  "یم", "ید", "ند", 
                  "مان", "تان", "شان"]
    
    prefixes = ["با", "بی", "نا"]
    
    # idea: avoid wrong cut with half space
    token = re.sub("|".join([f"^{st}" for st in stemming_b]), " ", token)
    token = re.sub("|".join([f"{pf}$" for pf in postfixes_b]), " ", token)
    
    if len(token) > 5: # idea: avoid wrong cut with char limit
        token = re.sub("|".join([f"{pf}$" for pf in postfixes_a]), " ", token)
        token = re.sub("|".join([f"{st}$" for st in stemming_e]), " ", token)
        token = re.sub("|".join([f"^{pf}" for pf in prefixes]), " ", token)
    
    fa_digits = "۱۲۳۴۵۶۷۸۹۰١٢٣٤٥٦٧٨٩٠"
    en_digits = "12345678901234567890"
    token = token.translate(str.maketrans(fa_digits, en_digits))

    token = re.sub("[ئي]", "ی", token)
    token = token.strip(".ء \u200b\u200c\u200d")
    return token

In [4]:
def tokenize(doc):
    tokens = np.array(doc.content.split())
    doc_id = np.full((len(tokens)), doc.id)
    return np.column_stack((tokens, doc_id))

In [5]:
%%time
# saved
tokenized_docs = []
for tokens_doc in df.apply(tokenize, axis=1):
    tokenized_docs.extend(tokens_doc)

CPU times: user 2.76 s, sys: 220 ms, total: 2.98 s
Wall time: 2.98 s


In [6]:
%%time
# saved
normalized_docs = []
for [token, doc] in tokenized_docs:
    token = normalize(normalize(token))
    normalized_docs.append([token, doc])

CPU times: user 1min 19s, sys: 600 ms, total: 1min 19s
Wall time: 1min 20s


In [7]:
%%time
# saved
tokens_docs = sorted(normalized_docs, key=lambda token_doc: token_doc[0])

CPU times: user 1.22 s, sys: 16 ms, total: 1.23 s
Wall time: 1.23 s


In [10]:
%%time
# saved
inverted_indexes = {}
for [token, t_docs] in groupby(tokens_docs, key=lambda tokens_doc: tokens_doc[0]):
    docs = sorted(map(lambda item: int(item[1]), t_docs))
    
    tf = {}
    for doc in docs:
        if doc in tf:
            tf[doc] += 1
        else:
            tf[doc] = 1
    
    if len(tf) > 1300:
        continue
    elif re.sub("\s+", "", token).isdigit() and len(re.sub("\s+", "", token)) < 4:
        continue
    elif re.match("^http|^https|^video", token) and len(token) > 15:
        continue
    elif len(token) < 2:
        continue
    else:
        inverted_indexes.update({ token: tf })
        
inverted_indexes_file = open("inverted_indexes", "ab")
pickle.dump(inverted_indexes, inverted_indexes_file)
inverted_indexes_file.close()

CPU times: user 1.99 s, sys: 23.9 ms, total: 2.01 s
Wall time: 2.03 s


In [5]:
inverted_indexes_file = open("inverted_indexes", "rb")
inverted_indexes = pickle.load(inverted_indexes_file)

In [6]:
%%time
# saved
def get_doc_length(doc_id):
    content = df.loc[df["id"] == doc_id, "content"].values[0]
    tokens = []
    for token in content.split():
        tokens.append(normalize(normalize(token)))
    
    length = 0
    for token in tokens:
        if token in inverted_indexes:
            length += inverted_indexes[token][doc_id] ** 2
    return math.sqrt(length)

df["length"] = df["id"].apply(get_doc_length)

CPU times: user 1min 16s, sys: 196 ms, total: 1min 17s
Wall time: 1min 17s


In [7]:
# saved
df.to_csv("IR_Spring2021_ph12_7k.csv", encoding='utf-8', index=False)

In [6]:
def merge_sort(input_list):
    if len(input_list) > 1:
        middle_index = len(input_list) // 2
        
        L = input_list[:middle_index]
        R = input_list[middle_index:]
 
        merge_sort(L)
        merge_sort(R)
 
        i = j = k = 0
 
        while i < len(L) and j < len(R):
            if L[i] < R[j]:
                input_list[k] = L[i]
                i += 1
            else:
                input_list[k] = R[j]
                j += 1
            k += 1
 
        while i < len(L):
            input_list[k] = L[i]
            i += 1
            k += 1
 
        while j < len(R):
            input_list[k] = R[j]
            j += 1
            k += 1
    return input_list

In [32]:
def search(query):
    k = 7
    with_heap = True
    with_champion = True
    
    tokens = []
    for token in query.split():
        tokens.append(normalize(normalize(token)))
    
    q_tf = {}
    for token in tokens:
        if token in q_tf:
            q_tf[token] += 1
        else:
            q_tf[token] = 1
            
    result = {}
    
    found_token = []
    for token in tokens:
        if token in inverted_indexes:
            found_token.append(token)
            doc_tf = inverted_indexes[token].items()
            idf = math.log(len(df) / len(doc_tf)) 
            q_tf_idf = (1 + math.log(q_tf[token])) * idf
            
            if with_champion:
                doc_tf = heapq.nlargest(k, doc_tf, key=lambda s: s[1])
            
            for doc, tf in doc_tf:              
                tf_idf = (1 + math.log(tf)) * idf
                if doc in result:
                    result[doc] += tf_idf * q_tf_idf
                else:
                    result.update({ doc: tf_idf * q_tf_idf })
              
    for doc in result:
        result[doc] /= df.loc[df["id"] == doc, "length"].values[0]
    
    res_df = df.loc[df["id"].isin(list(result.keys())), ["id", "url"]].copy()
    res_df["rank"] = res_df["id"].apply(lambda id: result[id])

    if with_heap:
        final_df = res_df.loc[res_df["rank"].isin(heapq.nlargest(k, list(res_df["rank"])))].sort_values(by=["rank"], ascending=False)
    else:
        final_df = res_df.loc[res_df["rank"].isin(merge_sort(list(res_df["rank"]))[-7:])].sort_values(by=["rank"], ascending=False)
    
    print(found_token)
    for index, row in final_df.iterrows():
            print("=" * 80)
            print(f"#Doc {row.id}")
            print(row.url)
            print(row["rank"])

In [38]:
%%time

# asasa = 'به گزارش ایسنا، فدراسیون بین المللی شنا قانونی را برای استفاده از فناوری ویدئو\nدر زیر آب تصویب کرده است که در توکیو ۲۰۲۰ استفاده می شود. از این تجهیزات برای\nتصمیمی که داور کنار استخر می گیرد استفاده می شود.\n\nبر این اساس در بازی های المپیک و مسابقات قهرمانی جهان از این تجهیزات استفاده\nمی شود.\n\nاین فناوری تصمیم گیری جدیدی در مورد شناگران در مسابقات ایجاد نمی کند فقط\nتصمیمات داور کنار استخر را تائید یا لغو می کند.\n\nاین فناوری برای تایید یا لغو تصمیمات داور کنار استخر استفاده خواهد شد. شناگران\nموظفند از این قوانین پیروی کنند.  \nاین فناوری بعد از تصویب در المپیک توکیو ۲۰۲۰ استفاده می شود.\n\nانتهای پیام\n\n'
# search(asasa)

search("تیم ملی تکواندو")
# search("قهرمانی پرسپولیس")
# search("تمرینات تیم تکواندو")

# search("قیمت‌گذاری کالاهای صادراتی")
# search("وام‌های اشتغال‌زایی عشایری و روستایی")

['تیم', 'تکواندو']
#Doc 62
https://www.isna.ir/news/99012716113/برنامه-های-پیش-روی-تکواندو-در-گرو-جلسه-پولادگر-با-کادر-فنی-تیم
1.6288028848677782
#Doc 818
https://www.isna.ir/news/99101612523/علی-نژاد-خطاب-به-ملی-پوشان-تکواندو-در-المپیک-مدال-بگیرید-تا
1.6103965304112329
#Doc 1291
https://www.isna.ir/news/98052411781/رییس-فدراسیون-تکواندو-پرداختن-به-منافع-شخصی-همدلی-در-تکواندو
1.4071763387606
#Doc 1130
https://www.isna.ir/news/98030100267/یوسف-کرمی-پولادگر-هم-در-این-ناکامی-مقصر-است-چرا-از-قهرمانان
1.1806699948918375
#Doc 650
https://www.isna.ir/news/99081308613/یوسف-کرمی-سازمان-لیگ-تکواندو-حقی-برای-ورزشکار-قائل-نیست
1.1188030834866889
#Doc 1288
https://www.isna.ir/news/98052210720/سرمربی-تکواندوی-نونهالان-این-بچه-ها-مدال-آور-المپیک-می-شوند
0.9754463750522372
#Doc 535
https://www.isna.ir/news/99071511812/رییس-فدراسیون-جهانی-تکواندو-اولویت-ما-حفظ-سلامت-ورزشکاران-است
0.913794347130121
CPU times: user 28.1 ms, sys: 0 ns, total: 28.1 ms
Wall time: 26 ms
